<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/SUPERVISTED/DBM_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Class that defines the behavior of the RBM
class RBM(object):
    
    def __init__(self, input_size, output_size, lr=1.0, batchsize=100):
        """
        m: Number of neurons in visible layer
        n: number of neurons in hidden layer
        """
        #Defining the hyperparameters
        self._input_size = input_size #Size of Visible
        self._output_size = output_size #Size of outp
        self.learning_rate = lr #The step used in gradient descent
        self.batchsize = batchsize #The size of how much data will be used for training per sub iteration
        
        #Initializing weights and biases as matrices full of zeroes
        self.w = tf.zeros([input_size, output_size], np.float32) #Creates and initializes the weights with 0
        self.hb = tf.zeros([output_size], np.float32) #Creates and initializes the hidden biases with 0
        self.vb = tf.zeros([input_size], np.float32) #Creates and initializes the visible biases with 0


    #Forward Pass
    def prob_h_given_v(self, visible, w, hb):
        #Sigmoid 
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)

    #Backward Pass
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    #Generate the sample probability
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))

    #Training method for the model
    def train(self, X, epochs=10):
               
        loss = []
        for epoch in range(epochs):
            #For each step/batch
            for start, end in zip(range(0, len(X), self.batchsize),range(self.batchsize,len(X), self.batchsize)):
                batch = X[start:end]
                    
                #Initialize with sample probabilities
                    
                h0 = self.sample_prob(self.prob_h_given_v(batch, self.w, self.hb))
                v1 = self.sample_prob(self.prob_v_given_h(h0, self.w, self.vb))
                h1 = self.prob_h_given_v(v1, self.w, self.hb)
                    
                #Create the Gradients
                positive_grad = tf.matmul(tf.transpose(batch), h0)
                negative_grad = tf.matmul(tf.transpose(v1), h1)
                    
                #Update learning rates 
                self.w = self.w + self.learning_rate *(positive_grad - negative_grad) / tf.dtypes.cast(tf.shape(batch)[0],tf.float32)
                self.vb = self.vb +  self.learning_rate * tf.reduce_mean(batch - v1, 0)
                self.hb = self.hb +  self.learning_rate * tf.reduce_mean(h0 - h1, 0)
                    
            #Find the error rate
            err = tf.reduce_mean(tf.square(batch - v1))
            print ('Epoch: %d' % epoch,'reconstruction error: %f' % err)
            loss.append(err)
                    
        return loss
    
    def rbm_reconstruct(self,X):
        h = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        reconstruct = tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.w)) + self.vb)
        return reconstruct

    def rbm_output(self, X):
        out = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        return out

In [3]:
(train_data, _), (test_data, _) = tf.keras.datasets.mnist.load_data()
train_data = train_data / 255.0
train_data = train_data.astype(np.float32)
train_data = np.reshape(train_data, (train_data.shape[0], 784))

test_data = test_data / np.float32(255)
test_data = np.reshape(test_data, (test_data.shape[0], 784))

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
RBM_hidden_sizes = [500, 200, 50]

input_data = train_data

rbm_list= []
input_size = input_data.shape[1]


#784 -> 500, 500 -> 200, 200 -> 50
#3개 RBM 네트워크를 stack 한다.
for i , size in enumerate(RBM_hidden_sizes):
    print("RBM: {:d} -> {:d}".format(input_size, size))
    rbm_list.append(RBM(input_size, size))
    input_size = size

for rbm in rbm_list:
    print('New RBM training...')
    rbm.train(tf.cast(input_data, tf.float32))
    input_data = rbm.rbm_output(input_data)


RBM: 784 -> 500
RBM: 500 -> 200
RBM: 200 -> 50
New RBM training...
Epoch: 0 reconstruction error: 0.058084
Epoch: 1 reconstruction error: 0.053617
Epoch: 2 reconstruction error: 0.049083
Epoch: 3 reconstruction error: 0.046656
Epoch: 4 reconstruction error: 0.046165
Epoch: 5 reconstruction error: 0.044353
Epoch: 6 reconstruction error: 0.043565
Epoch: 7 reconstruction error: 0.043504
Epoch: 8 reconstruction error: 0.042443
Epoch: 9 reconstruction error: 0.042670
New RBM training...
Epoch: 0 reconstruction error: 0.029680
Epoch: 1 reconstruction error: 0.026589
Epoch: 2 reconstruction error: 0.025454
Epoch: 3 reconstruction error: 0.024009
Epoch: 4 reconstruction error: 0.021566
Epoch: 5 reconstruction error: 0.022320
Epoch: 6 reconstruction error: 0.021863
Epoch: 7 reconstruction error: 0.022237
Epoch: 8 reconstruction error: 0.021484
Epoch: 9 reconstruction error: 0.022986
New RBM training...
Epoch: 0 reconstruction error: 0.053768
Epoch: 1 reconstruction error: 0.046305
Epoch: 2 reco